<a href="https://colab.research.google.com/github/AnwarSR-DA/final-project-ds49-group3/blob/main/eda%20%26%20preprocessing/eda_%26_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Connect to GitHub

## 0.1 Import Libraries

## 0.2 Load Dataset

# 1. Exploratory Data Analysis

## 1.1 Check Null Values

## 1.2 Check Duplicate Data

## 1.3 Check Unique Values

### 1.3.1 Check Data Shape

### 1.3.2 Check Data Type

### 1.3.3 Check Data Stats

## 1.4 Outlier Detection

## 1.5 Correlation Heatmap

## 1.6 Visualisasi Target Delivery Time

# 2. Data Preprocessing

## 2.1 Feature Engineering: Hitung `delivery_duration_minutes`

## 2.2 Drop kolom tidak relevan

## 2.3 Handle Missing Values

## 2.4 Encoding (jika perlu)

## 2.5 Feature Scaling

## 2.6 Save cleaned data untuk modeling